In [183]:
import gymnasium as gym
import numpy as np
import d3rlpy
import pickle

import matplotlib.pyplot as plt
from env.atari.represented_atari_game import GymCompatWrapper2
from online_main import OneHotWrapper

hyperparams = {
        "env": "RepresentedPong-v0", #"CartPole-v0", "MountainCar-v0", "FrozenLake-v1", "CliffWalking-v0", "RepresentedPong-v0"
        "seed": 42069,
        "n_episodes": 300,
        "max_episode_len": 200, # Around 10h per 100k steps in Leviathan server
        "eps": 0.1,  # epsilon for exploration
        "n_exp": 5,
        "n_pretrain_eps": 30,
        "n_online_eps": 270, #10-5990 for mountainCar, 30-120 for CartPole
        "gpu": True, # True if use GPU to train with d3rlpy
        "buffer_size": 100000, #Test with 100k, 200k, 500k. 1M might be too much
        "data_path": None,#'data/CartPole_Qwen2.5-7B-Instruct_Neps_10_20250406040150.pkl',
        "model_path": None,#'d3rlpy_loss/DoubleDQN_online_20250331153346/model_600000.d3',
        "batch_size":256, #Test smaller batch size: 32, 64. May be noisier
        "learning_rate":5e-5,
        "gamma":0.99,
        "target_update_interval":1000 #Test with 1k, 2k, 5k
    }

assert hyperparams["n_episodes"]==hyperparams["n_pretrain_eps"]+hyperparams["n_online_eps"], "Check n_episodes=n_pretrain_eps+n_online_eps"

if "Represented" in hyperparams["env"]:
    env = GymCompatWrapper2(gym.make(hyperparams["env"]))
    eval_env = GymCompatWrapper2(gym.make(hyperparams["env"]))
elif isinstance(gym.make(hyperparams["env"]).observation_space, gym.spaces.Discrete):
    env = OneHotWrapper(gym.make(hyperparams["env"]))
    eval_env = OneHotWrapper(gym.make(hyperparams["env"]))
else:
    env = gym.make(hyperparams["env"])
    eval_env = gym.make(hyperparams["env"])

# fix seed
d3rlpy.seed(hyperparams["seed"])
d3rlpy.envs.seed_env(env, hyperparams["seed"])
d3rlpy.envs.seed_env(eval_env, hyperparams["seed"])

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
/opt/miniconda3/envs/llamagym/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.labels to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.labels` for environment variables or `env.get_wrapper_attr('labels')` that will search the reminding wrappers.
  logger.warn(
/opt/miniconda3/envs/llamagym/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.spec to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.spec` for environment variables or `env.get_wrapper_attr('spec')` that will search the reminding wrappers.
  logger.warn(
/opt/miniconda3/envs/llamagym/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:175: UserWarning: WARN: The default seed argument in `Env.reset` should be `None`, otherwise the envir

### Static/pretrain data

In [184]:
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-7B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250413234248.pkl", 'rb') as file: #Pendulum
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-7B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250412171921.pkl", 'rb') as file: #CliffWalking
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-7B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250412075104.pkl", 'rb') as file: #FrozenLake
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-7B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250411000858.pkl", 'rb') as file: #FrozenLakeBug
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-7B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250410211529.pkl", 'rb') as file: #CartPole
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-7B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250409023954E.pkl", 'rb') as file: #CartPoleE
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-7B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250415095844.pkl", 'rb') as file: #MountainCar
with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-7B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250420103044.pkl", 'rb') as file: #RepresentedPong
    Qwen_7B_dataset = pickle.load(file)
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-32B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250414014508.pkl", 'rb') as file: #Pendulum
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-32B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250415065446.pkl", 'rb') as file: #CliffWalking
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-32B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250412120230.pkl", 'rb') as file: #FrozenLake
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-32B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250411030422.pkl", 'rb') as file: #FrozenLakeBug
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-32B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250412032827.pkl", 'rb') as file: #CartPole
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-32B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250409124533E.pkl", 'rb') as file: #CartPoleE
# with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-32B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250413081613.pkl", 'rb') as file: #MountainCar
with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-32B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250420162547.pkl", 'rb') as file: #RepresentedPong
    Qwen_32B_dataset = pickle.load(file)

Qwen_32B_rewards = []
for i in range(hyperparams["n_pretrain_eps"]):
    Qwen_32B_rewards.append(Qwen_32B_dataset.episodes[i].compute_return())
Qwen_7B_rewards = []
for i in range(hyperparams["n_pretrain_eps"]):
    Qwen_7B_rewards.append(Qwen_7B_dataset.episodes[i].compute_return())

Qwen_7B = Qwen_7B_rewards + hyperparams["n_online_eps"] * [np.mean(Qwen_7B_rewards)]
Qwen_32B = Qwen_32B_rewards + hyperparams["n_online_eps"] * [np.mean(Qwen_32B_rewards)]

print("Qwen_32B: ", np.mean(Qwen_7B_rewards))
print("Qwen_7B: ", np.mean(Qwen_32B_rewards))

Qwen_32B:  -4.966666666666667
Qwen_7B:  -2.966666666666667


In [185]:
try:
    # with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-7B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250421120400SFT.pkl", 'rb') as file: #FrozenLake
    with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-7B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250421175530SFT.pkl", 'rb') as file: #CliffWalking
    # with open(f"data/{hyperparams['env'].split('-')[0]}_Qwen2.5-7B-Instruct_Neps_{hyperparams['n_pretrain_eps']}_20250422210707SFT.pkl", 'rb') as file: #Pendulum
        Qwen_7B_SFT_dataset = pickle.load(file)

    Qwen_7B_SFT_rewards = []
    for i in range(hyperparams["n_pretrain_eps"]):
        Qwen_7B_SFT_rewards.append(Qwen_7B_SFT_dataset.episodes[i].compute_return())
    Qwen_7B_SFT = Qwen_7B_SFT_rewards + hyperparams["n_online_eps"] * [np.mean(Qwen_7B_SFT_rewards)]
    print("Qwen_7B_SFT: ", np.mean(Qwen_7B_SFT_rewards))
except:
    Qwen_7B_SFT = None
try:
    with open(f"data/{hyperparams['env'].split('-')[0]}_DeepSeek-R1-Distill-Qwen-7B_Neps_{hyperparams['n_pretrain_eps']}_20250419172821.pkl", 'rb') as file: #FrozenLake
        DS_7B_dataset = pickle.load(file)
    with open(f"data/{hyperparams['env'].split('-')[0]}_DeepSeek-R1-Distill-Qwen-14B_Neps_{hyperparams['n_pretrain_eps']}_20250422000525.pkl", 'rb') as file: #FrozenLake
        DS_14B_dataset = pickle.load(file)
    DS_7B_rewards = []
    for i in range(hyperparams["n_pretrain_eps"]):
        DS_7B_rewards.append(DS_7B_dataset.episodes[i].compute_return())
    DS_14B_rewards = []
    for i in range(hyperparams["n_pretrain_eps"]):
        DS_14B_rewards.append(DS_14B_dataset.episodes[i].compute_return())

    DS_7B = DS_7B_rewards + hyperparams["n_online_eps"] * [np.mean(DS_7B_rewards)]
    DS_14B = DS_14B_rewards + hyperparams["n_online_eps"] * [np.mean(DS_14B_rewards)]

    print("DS_7B: ", np.mean(DS_7B_rewards))
    print("DS_14B: ", np.mean(DS_14B_rewards))
except:
    DS_7B = None
    DS_14B = None

### Generate and evaluate random baseline

In [186]:
# Evaluate the random policy manually
random_rewards = []
for _ in range(hyperparams["n_pretrain_eps"]):
	obs, _ = env.reset()
	done = False
	total_reward = 0
	count = 0
	while not done:
		action = env.action_space.sample()
		obs, reward, done, _, _ = env.step(action)
		total_reward += reward
		count += 1
		if count >= hyperparams["max_episode_len"]:
			break
	random_rewards.append(total_reward)

mean_random = np.ones(hyperparams["n_pretrain_eps"] + hyperparams["n_online_eps"]) * np.mean(random_rewards)
# Print the average reward of the random policy
print(f"Average reward for random policy: {mean_random[0]}")

/opt/miniconda3/envs/llamagym/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:213: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/opt/miniconda3/envs/llamagym/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: int64
  logger.warn(
/opt/miniconda3/envs/llamagym/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


Average reward for random policy: -4.4


# Plot

In [187]:
def extract_data(hyperparams, Qwen_7B, Qwen_32B, DS_7B, DS_14B, Qwen_7B_SFT):
    with open(f"data/cache_{hyperparams['env'].split('-')[0]}_Neps_10.pkl", 'rb') as file:
        cache10 = pickle.load(file)
    with open(f"data/cache_{hyperparams['env'].split('-')[0]}_Neps_20.pkl", 'rb') as file:
        cache20 = pickle.load(file)
    with open(f"data/cache_{hyperparams['env'].split('-')[0]}_Neps_30.pkl", 'rb') as file:
        cache30 = pickle.load(file)
    with open(f'data/cache_{hyperparams["env"].split("-")[0]}_on_policy_pretrain_exp.pkl', 'rb') as file:
        on_policy_pretrain_cache = pickle.load(file)
    with open(f'data/cache_{hyperparams["env"].split("-")[0]}_on_policy_pretrain_exp_rand.pkl', 'rb') as file:
        rand_pretrain_cache = pickle.load(file)
    try:
        with open(f"data/cache_{hyperparams['env'].split('-')[0]}_Neps_10SFT.pkl", 'rb') as file:
            cache10SFT = pickle.load(file)
        with open(f"data/cache_{hyperparams['env'].split('-')[0]}_Neps_20SFT.pkl", 'rb') as file:
            cache20SFT = pickle.load(file)
        with open(f"data/cache_{hyperparams['env'].split('-')[0]}_Neps_30SFT.pkl", 'rb') as file:
            cache30SFT = pickle.load(file)
        pretrain_7b_1000_pre_10SFT_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_7b_3000_pre_10SFT_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_7b_1000_pre_20SFT_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_7b_3000_pre_20SFT_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_7b_1000_pre_30SFT_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_7b_3000_pre_30SFT_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))

        for i in range(hyperparams["n_exp"]):
            for j in range(hyperparams["n_episodes"]-10):
                pretrain_7b_1000_pre_10SFT_returns[i][10+j] = cache10SFT[f"pretrain_7b_1000_{i}"][j]
                pretrain_7b_3000_pre_10SFT_returns[i][10+j] = cache10SFT[f"pretrain_7b_3000_{i}"][j]

            for j in range(hyperparams["n_episodes"]-20):
                pretrain_7b_1000_pre_20SFT_returns[i][20+j] = cache20SFT[f"pretrain_7b_1000_{i}"][j]
                pretrain_7b_3000_pre_20SFT_returns[i][20+j] = cache20SFT[f"pretrain_7b_3000_{i}"][j]

            for j in range(hyperparams["n_episodes"]-30):
                pretrain_7b_1000_pre_30SFT_returns[i][30+j] = cache30SFT[f"pretrain_7b_1000_{i}"][j]
                pretrain_7b_3000_pre_30SFT_returns[i][30+j] = cache30SFT[f"pretrain_7b_3000_{i}"][j]
        mean_pretrain_7b_1000_pre_10SFT = np.mean(pretrain_7b_1000_pre_10SFT_returns, axis = 0)
        std_pretrain_7b_1000_pre_10SFT = np.std(pretrain_7b_1000_pre_10SFT_returns, axis = 0)
        mean_pretrain_7b_3000_pre_10SFT = np.mean(pretrain_7b_3000_pre_10SFT_returns, axis = 0)
        std_pretrain_7b_3000_pre_10SFT = np.std(pretrain_7b_3000_pre_10SFT_returns, axis = 0)
        mean_pretrain_7b_1000_pre_10SFT[:10] = Qwen_7B_SFT[:10]
        mean_pretrain_7b_3000_pre_10SFT[:10] = Qwen_7B_SFT[:10]

        mean_pretrain_7b_1000_pre_20SFT = np.mean(pretrain_7b_1000_pre_20SFT_returns, axis = 0)
        std_pretrain_7b_1000_pre_20SFT = np.std(pretrain_7b_1000_pre_20SFT_returns, axis = 0)
        mean_pretrain_7b_3000_pre_20SFT = np.mean(pretrain_7b_3000_pre_20SFT_returns, axis = 0)
        std_pretrain_7b_3000_pre_20SFT = np.std(pretrain_7b_3000_pre_20SFT_returns, axis = 0)
        mean_pretrain_7b_1000_pre_20SFT[:20] = Qwen_7B_SFT[:20]
        mean_pretrain_7b_3000_pre_20SFT[:20] = Qwen_7B_SFT[:20]

        mean_pretrain_7b_1000_pre_30SFT = np.mean(pretrain_7b_1000_pre_30SFT_returns, axis = 0)
        std_pretrain_7b_1000_pre_30SFT = np.std(pretrain_7b_1000_pre_30SFT_returns, axis = 0)
        mean_pretrain_7b_3000_pre_30SFT = np.mean(pretrain_7b_3000_pre_30SFT_returns, axis = 0)
        std_pretrain_7b_3000_pre_30SFT = np.std(pretrain_7b_3000_pre_30SFT_returns, axis = 0)
        mean_pretrain_7b_1000_pre_30SFT[:30] = Qwen_7B_SFT[:30]
        mean_pretrain_7b_3000_pre_30SFT[:30] = Qwen_7B_SFT[:30]
        out_dict_SFT = {
            "mean_pretrain_7b_1000_pre_10SFT": mean_pretrain_7b_1000_pre_10SFT,
            "std_pretrain_7b_1000_pre_10SFT": std_pretrain_7b_1000_pre_10SFT,
            "mean_pretrain_7b_3000_pre_10SFT": mean_pretrain_7b_3000_pre_10SFT,
            "std_pretrain_7b_3000_pre_10SFT": std_pretrain_7b_3000_pre_10SFT,
            "mean_pretrain_7b_1000_pre_20SFT": mean_pretrain_7b_1000_pre_20SFT,
            "std_pretrain_7b_1000_pre_20SFT": std_pretrain_7b_1000_pre_20SFT,
            "mean_pretrain_7b_3000_pre_20SFT": mean_pretrain_7b_3000_pre_20SFT,
            "std_pretrain_7b_3000_pre_20SFT": std_pretrain_7b_3000_pre_20SFT,
            "mean_pretrain_7b_1000_pre_30SFT": mean_pretrain_7b_1000_pre_30SFT,
            "std_pretrain_7b_1000_pre_30SFT": std_pretrain_7b_1000_pre_30SFT,
            "mean_pretrain_7b_3000_pre_30SFT": mean_pretrain_7b_3000_pre_30SFT,
            "std_pretrain_7b_3000_pre_30SFT": std_pretrain_7b_3000_pre_30SFT
        }
    except:
        pass
    try:
        with open(f"data/cache_{hyperparams['env'].split('-')[0]}_Neps_10DS.pkl", 'rb') as file:
            cache10DS = pickle.load(file)
        with open(f"data/cache_{hyperparams['env'].split('-')[0]}_Neps_20DS.pkl", 'rb') as file:
            cache20DS = pickle.load(file)
        with open(f"data/cache_{hyperparams['env'].split('-')[0]}_Neps_30DS.pkl", 'rb') as file:
            cache30DS = pickle.load(file)
        pretrain_7b_1000_pre_10DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_7b_3000_pre_10DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_14b_1000_pre_10DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_14b_3000_pre_10DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_7b_1000_pre_20DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_7b_3000_pre_20DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_14b_1000_pre_20DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_14b_3000_pre_20DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_7b_1000_pre_30DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_7b_3000_pre_30DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_14b_1000_pre_30DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        pretrain_14b_3000_pre_30DS_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
        for i in range(hyperparams["n_exp"]):
            for j in range(hyperparams["n_episodes"]-10):
                pretrain_7b_1000_pre_10DS_returns[i][10+j] = cache10DS[f"pretrain_7b_1000_{i}"][j]
                pretrain_14b_1000_pre_10DS_returns[i][10+j] = cache10DS[f"pretrain_32b_1000_{i}"][j]
                pretrain_7b_3000_pre_10DS_returns[i][10+j] = cache10DS[f"pretrain_7b_3000_{i}"][j]
                pretrain_14b_3000_pre_10DS_returns[i][10+j] = cache10DS[f"pretrain_32b_3000_{i}"][j]

            for j in range(hyperparams["n_episodes"]-20):
                pretrain_7b_1000_pre_20DS_returns[i][20+j] = cache20DS[f"pretrain_7b_1000_{i}"][j]
                pretrain_14b_1000_pre_20DS_returns[i][20+j] = cache20DS[f"pretrain_32b_1000_{i}"][j]
                pretrain_7b_3000_pre_20DS_returns[i][20+j] = cache20DS[f"pretrain_7b_3000_{i}"][j]
                pretrain_14b_3000_pre_20DS_returns[i][20+j] = cache20DS[f"pretrain_32b_3000_{i}"][j]

            for j in range(hyperparams["n_episodes"]-30):
                pretrain_7b_1000_pre_30DS_returns[i][30+j] = cache30DS[f"pretrain_7b_1000_{i}"][j]
                pretrain_14b_1000_pre_30DS_returns[i][30+j] = cache30DS[f"pretrain_32b_1000_{i}"][j]
                pretrain_7b_3000_pre_30DS_returns[i][30+j] = cache30DS[f"pretrain_7b_3000_{i}"][j]
                pretrain_14b_3000_pre_30DS_returns[i][30+j] = cache30DS[f"pretrain_32b_3000_{i}"][j]
        mean_pretrain_7b_1000_pre_10DS = np.mean(pretrain_7b_1000_pre_10DS_returns, axis = 0)
        std_pretrain_7b_1000_pre_10DS = np.std(pretrain_7b_1000_pre_10DS_returns, axis = 0)
        mean_pretrain_7b_3000_pre_10DS = np.mean(pretrain_7b_3000_pre_10DS_returns, axis = 0)
        std_pretrain_7b_3000_pre_10DS = np.std(pretrain_7b_3000_pre_10DS_returns, axis = 0)
        mean_pretrain_14b_1000_pre_10DS = np.mean(pretrain_14b_1000_pre_10DS_returns, axis = 0)
        std_pretrain_14b_1000_pre_10DS = np.std(pretrain_14b_1000_pre_10DS_returns, axis = 0)
        mean_pretrain_14b_3000_pre_10DS = np.mean(pretrain_14b_3000_pre_10DS_returns, axis = 0)
        std_pretrain_14b_3000_pre_10DS = np.std(pretrain_14b_3000_pre_10DS_returns, axis = 0)
        mean_pretrain_14b_1000_pre_10DS[:10] = DS_14B[:10]
        mean_pretrain_7b_1000_pre_10DS[:10] = DS_7B[:10]
        mean_pretrain_14b_3000_pre_10DS[:10] = DS_14B[:10]
        mean_pretrain_7b_3000_pre_10DS[:10] = DS_7B[:10]

        mean_pretrain_7b_1000_pre_20DS = np.mean(pretrain_7b_1000_pre_20DS_returns, axis = 0)
        std_pretrain_7b_1000_pre_20DS = np.std(pretrain_7b_1000_pre_20DS_returns, axis = 0)
        mean_pretrain_7b_3000_pre_20DS = np.mean(pretrain_7b_3000_pre_20DS_returns, axis = 0)
        std_pretrain_7b_3000_pre_20DS = np.std(pretrain_7b_3000_pre_20DS_returns, axis = 0)
        mean_pretrain_14b_1000_pre_20DS = np.mean(pretrain_14b_1000_pre_20DS_returns, axis = 0)
        std_pretrain_14b_1000_pre_20DS = np.std(pretrain_14b_1000_pre_20DS_returns, axis = 0)
        mean_pretrain_14b_3000_pre_20DS = np.mean(pretrain_14b_3000_pre_20DS_returns, axis = 0)
        std_pretrain_14b_3000_pre_20DS = np.std(pretrain_14b_3000_pre_20DS_returns, axis = 0)
        mean_pretrain_14b_1000_pre_20DS[:20] = DS_14B[:20]
        mean_pretrain_7b_1000_pre_20DS[:20] = DS_7B[:20]
        mean_pretrain_14b_3000_pre_20DS[:20] = DS_14B[:20]
        mean_pretrain_7b_3000_pre_20DS[:20] = DS_7B[:20]

        mean_pretrain_7b_1000_pre_30DS = np.mean(pretrain_7b_1000_pre_30DS_returns, axis = 0)
        std_pretrain_7b_1000_pre_30DS = np.std(pretrain_7b_1000_pre_30DS_returns, axis = 0)
        mean_pretrain_7b_3000_pre_30DS = np.mean(pretrain_7b_3000_pre_30DS_returns, axis = 0)
        std_pretrain_7b_3000_pre_30DS = np.std(pretrain_7b_3000_pre_30DS_returns, axis = 0)
        mean_pretrain_14b_1000_pre_30DS = np.mean(pretrain_14b_1000_pre_30DS_returns, axis = 0)
        std_pretrain_14b_1000_pre_30DS = np.std(pretrain_14b_1000_pre_30DS_returns, axis = 0)
        mean_pretrain_14b_3000_pre_30DS = np.mean(pretrain_14b_3000_pre_30DS_returns, axis = 0)
        std_pretrain_14b_3000_pre_30DS = np.std(pretrain_14b_3000_pre_30DS_returns, axis = 0)
        mean_pretrain_14b_1000_pre_30DS[:30] = DS_14B[:30]
        mean_pretrain_7b_1000_pre_30DS[:30] = DS_7B[:30]
        mean_pretrain_14b_3000_pre_30DS[:30] = DS_14B[:30]
        mean_pretrain_7b_3000_pre_30DS[:30] = DS_7B[:30]
        out_dict_DS = {
            "mean_pretrain_7b_1000_pre_10DS": mean_pretrain_7b_1000_pre_10DS,
            "std_pretrain_7b_1000_pre_10DS": std_pretrain_7b_1000_pre_10DS,
            "mean_pretrain_7b_3000_pre_10DS": mean_pretrain_7b_3000_pre_10DS,
            "std_pretrain_7b_3000_pre_10DS": std_pretrain_7b_3000_pre_10DS,
            "mean_pretrain_14b_1000_pre_10DS": mean_pretrain_14b_1000_pre_10DS,
            "std_pretrain_14b_1000_pre_10DS": std_pretrain_14b_1000_pre_10DS,
            "mean_pretrain_14b_3000_pre_10DS": mean_pretrain_14b_3000_pre_10DS,
            "std_pretrain_14b_3000_pre_10DS": std_pretrain_14b_3000_pre_10DS,
            "mean_pretrain_7b_1000_pre_20DS": mean_pretrain_7b_1000_pre_20DS,
            "std_pretrain_7b_1000_pre_20DS": std_pretrain_7b_1000_pre_20DS,
            "mean_pretrain_7b_3000_pre_20DS": mean_pretrain_7b_3000_pre_20DS,
            "std_pretrain_7b_3000_pre_20DS": std_pretrain_7b_3000_pre_20DS,
            "mean_pretrain_14b_1000_pre_20DS": mean_pretrain_14b_1000_pre_20DS,
            "std_pretrain_14b_1000_pre_20DS": std_pretrain_14b_1000_pre_20DS,
            "mean_pretrain_14b_3000_pre_20DS": mean_pretrain_14b_3000_pre_20DS,
            "std_pretrain_14b_3000_pre_20DS": std_pretrain_14b_3000_pre_20DS,
            "mean_pretrain_7b_1000_pre_30DS": mean_pretrain_7b_1000_pre_30DS,
            "std_pretrain_7b_1000_pre_30DS": std_pretrain_7b_1000_pre_30DS,
            "mean_pretrain_7b_3000_pre_30DS": mean_pretrain_7b_3000_pre_30DS,
            "std_pretrain_7b_3000_pre_30DS": std_pretrain_7b_3000_pre_30DS,
            "mean_pretrain_14b_1000_pre_30DS": mean_pretrain_14b_1000_pre_30DS,
            "std_pretrain_14b_1000_pre_30DS": std_pretrain_14b_1000_pre_30DS,
            "mean_pretrain_14b_3000_pre_30DS": mean_pretrain_14b_3000_pre_30DS,
            "std_pretrain_14b_3000_pre_30DS": std_pretrain_14b_3000_pre_30DS
        }
    except:
        pass
    online_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    mix_7b_pre_10_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    mix_32b_pre_10_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_7b_1000_pre_10_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_32b_1000_pre_10_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_7b_3000_pre_10_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_32b_3000_pre_10_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    on_pol_pretrain_1000_pre_10_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    on_pol_pretrain_3000_pre_10_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    rand_pretrain_1000_pre_10_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    rand_pretrain_3000_pre_10_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))

    mix_7b_pre_20_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    mix_32b_pre_20_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_7b_1000_pre_20_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_32b_1000_pre_20_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_7b_3000_pre_20_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_32b_3000_pre_20_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    on_pol_pretrain_1000_pre_20_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    on_pol_pretrain_3000_pre_20_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    rand_pretrain_1000_pre_20_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    rand_pretrain_3000_pre_20_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))

    mix_7b_pre_30_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    mix_32b_pre_30_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_7b_1000_pre_30_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_32b_1000_pre_30_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_7b_3000_pre_30_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    pretrain_32b_3000_pre_30_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    on_pol_pretrain_1000_pre_30_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    on_pol_pretrain_3000_pre_30_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    rand_pretrain_1000_pre_30_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))
    rand_pretrain_3000_pre_30_returns = np.zeros((hyperparams["n_exp"], hyperparams["n_episodes"]))

    for i in range(hyperparams["n_exp"]):
        # online_returns[i] = cache10[f"online_{i}"]
        online_returns[i] = cache10[f"online_{i}"][:hyperparams["n_episodes"]] #Quick-fix for MountainCar
        on_pol_pretrain_1000_pre_10_returns[i] = on_policy_pretrain_cache[f'pretrain_10_eps_1000_steps_{i}']
        on_pol_pretrain_3000_pre_10_returns[i] = on_policy_pretrain_cache[f'pretrain_10_eps_3000_steps_{i}']
        on_pol_pretrain_1000_pre_20_returns[i] = on_policy_pretrain_cache[f'pretrain_20_eps_1000_steps_{i}']
        on_pol_pretrain_3000_pre_20_returns[i] = on_policy_pretrain_cache[f'pretrain_20_eps_3000_steps_{i}']
        on_pol_pretrain_1000_pre_30_returns[i] = on_policy_pretrain_cache[f'pretrain_30_eps_1000_steps_{i}']
        on_pol_pretrain_3000_pre_30_returns[i] = on_policy_pretrain_cache[f'pretrain_30_eps_3000_steps_{i}']

        rand_pretrain_1000_pre_10_returns[i] = rand_pretrain_cache[f'pretrain_10_eps_1000_steps_{i}_rand']
        rand_pretrain_3000_pre_10_returns[i] = rand_pretrain_cache[f'pretrain_10_eps_3000_steps_{i}_rand']
        rand_pretrain_1000_pre_20_returns[i] = rand_pretrain_cache[f'pretrain_20_eps_1000_steps_{i}_rand']
        rand_pretrain_3000_pre_20_returns[i] = rand_pretrain_cache[f'pretrain_20_eps_3000_steps_{i}_rand']
        rand_pretrain_1000_pre_30_returns[i] = rand_pretrain_cache[f'pretrain_30_eps_1000_steps_{i}_rand']
        rand_pretrain_3000_pre_30_returns[i] = rand_pretrain_cache[f'pretrain_30_eps_3000_steps_{i}_rand']
        for j in range(hyperparams["n_episodes"]-10):
            mix_7b_pre_10_returns[i][10+j] = cache10[f"finetune_7b_{i}"][j]
            mix_32b_pre_10_returns[i][10+j] = cache10[f"finetune_32b_{i}"][j]
            pretrain_7b_1000_pre_10_returns[i][10+j] = cache10[f"pretrain_7b_1000_{i}"][j]
            pretrain_32b_1000_pre_10_returns[i][10+j] = cache10[f"pretrain_32b_1000_{i}"][j]
            pretrain_7b_3000_pre_10_returns[i][10+j] = cache10[f"pretrain_7b_3000_{i}"][j]
            pretrain_32b_3000_pre_10_returns[i][10+j] = cache10[f"pretrain_32b_3000_{i}"][j]

        for j in range(hyperparams["n_episodes"]-20):
            mix_7b_pre_20_returns[i][20+j] = cache20[f"finetune_7b_{i}"][j]
            mix_32b_pre_20_returns[i][20+j] = cache20[f"finetune_32b_{i}"][j]
            pretrain_7b_1000_pre_20_returns[i][20+j] = cache20[f"pretrain_7b_1000_{i}"][j]
            pretrain_32b_1000_pre_20_returns[i][20+j] = cache20[f"pretrain_32b_1000_{i}"][j]
            pretrain_7b_3000_pre_20_returns[i][20+j] = cache20[f"pretrain_7b_3000_{i}"][j]
            pretrain_32b_3000_pre_20_returns[i][20+j] = cache20[f"pretrain_32b_3000_{i}"][j]

        for j in range(hyperparams["n_episodes"]-30):
            mix_7b_pre_30_returns[i][30+j] = cache30[f"finetune_7b_{i}"][j]
            mix_32b_pre_30_returns[i][30+j] = cache30[f"finetune_32b_{i}"][j]
            pretrain_7b_1000_pre_30_returns[i][30+j] = cache30[f"pretrain_7b_1000_{i}"][j]
            pretrain_32b_1000_pre_30_returns[i][30+j] = cache30[f"pretrain_32b_1000_{i}"][j]
            pretrain_7b_3000_pre_30_returns[i][30+j] = cache30[f"pretrain_7b_3000_{i}"][j]
            pretrain_32b_3000_pre_30_returns[i][30+j] = cache30[f"pretrain_32b_3000_{i}"][j]

    x = range(hyperparams["n_episodes"])

    mean_mix_32b_pre_10 = np.mean(mix_32b_pre_10_returns, axis = 0)
    std_mix_32b_pre_10 = np.std(mix_32b_pre_10_returns, axis = 0)
    mean_mix_7b_pre_10 = np.mean(mix_7b_pre_10_returns, axis = 0)
    std_mix_7b_pre_10 = np.std(mix_7b_pre_10_returns, axis = 0)
    mean_onl = np.mean(online_returns, axis = 0)
    std_onl = np.std(online_returns, axis = 0)
    mean_pretrain_32b_1000_pre_10 = np.mean(pretrain_32b_1000_pre_10_returns, axis = 0)
    std_pretrain_32b_1000_pre_10 = np.std(pretrain_32b_1000_pre_10_returns, axis = 0)
    mean_pretrain_7b_1000_pre_10 = np.mean(pretrain_7b_1000_pre_10_returns, axis = 0)
    std_pretrain_7b_1000_pre_10 = np.std(pretrain_7b_1000_pre_10_returns, axis = 0)
    mean_pretrain_32b_3000_pre_10 = np.mean(pretrain_32b_3000_pre_10_returns, axis = 0)
    std_pretrain_32b_3000_pre_10 = np.std(pretrain_32b_3000_pre_10_returns, axis = 0)
    mean_pretrain_7b_3000_pre_10 = np.mean(pretrain_7b_3000_pre_10_returns, axis = 0)
    std_pretrain_7b_3000_pre_10 = np.std(pretrain_7b_3000_pre_10_returns, axis = 0)
    mean_on_pol_pretrain_1000_pre_10 = np.mean(on_pol_pretrain_1000_pre_10_returns, axis = 0)
    std_on_pol_pretrain_1000_pre_10 = np.std(on_pol_pretrain_1000_pre_10_returns, axis = 0)
    mean_on_pol_pretrain_3000_pre_10 = np.mean(on_pol_pretrain_3000_pre_10_returns, axis = 0)
    std_on_pol_pretrain_3000_pre_10 = np.std(on_pol_pretrain_3000_pre_10_returns, axis = 0)
    mean_rand_pretrain_1000_pre_10 = np.mean(rand_pretrain_1000_pre_10_returns, axis = 0)
    std_rand_pretrain_1000_pre_10 = np.std(rand_pretrain_1000_pre_10_returns, axis = 0)
    mean_rand_pretrain_3000_pre_10 = np.mean(rand_pretrain_3000_pre_10_returns, axis = 0)
    std_rand_pretrain_3000_pre_10 = np.std(rand_pretrain_3000_pre_10_returns, axis = 0)

    mean_mix_7b_pre_10[:10] = Qwen_7B[:10]
    mean_mix_32b_pre_10[:10] = Qwen_32B[:10]
    mean_pretrain_32b_1000_pre_10[:10] = Qwen_32B[:10]
    mean_pretrain_7b_1000_pre_10[:10] = Qwen_7B[:10]
    mean_pretrain_32b_3000_pre_10[:10] = Qwen_32B[:10]
    mean_pretrain_7b_3000_pre_10[:10] = Qwen_7B[:10]

    mean_mix_32b_pre_20 = np.mean(mix_32b_pre_20_returns, axis = 0)
    std_mix_32b_pre_20 = np.std(mix_32b_pre_20_returns, axis = 0)
    mean_mix_7b_pre_20 = np.mean(mix_7b_pre_20_returns, axis = 0)
    std_mix_7b_pre_20 = np.std(mix_7b_pre_20_returns, axis = 0)
    mean_pretrain_32b_1000_pre_20 = np.mean(pretrain_32b_1000_pre_20_returns, axis = 0)
    std_pretrain_32b_1000_pre_20 = np.std(pretrain_32b_1000_pre_20_returns, axis = 0)
    mean_pretrain_7b_1000_pre_20 = np.mean(pretrain_7b_1000_pre_20_returns, axis = 0)
    std_pretrain_7b_1000_pre_20 = np.std(pretrain_7b_1000_pre_20_returns, axis = 0)
    mean_pretrain_32b_3000_pre_20 = np.mean(pretrain_32b_3000_pre_20_returns, axis = 0)
    std_pretrain_32b_3000_pre_20 = np.std(pretrain_32b_3000_pre_20_returns, axis = 0)
    mean_pretrain_7b_3000_pre_20 = np.mean(pretrain_7b_3000_pre_20_returns, axis = 0)
    std_pretrain_7b_3000_pre_20 = np.std(pretrain_7b_3000_pre_20_returns, axis = 0)
    mean_on_pol_pretrain_1000_pre_20 = np.mean(on_pol_pretrain_1000_pre_20_returns, axis = 0)
    std_on_pol_pretrain_1000_pre_20 = np.std(on_pol_pretrain_1000_pre_20_returns, axis = 0)
    mean_on_pol_pretrain_3000_pre_20 = np.mean(on_pol_pretrain_3000_pre_20_returns, axis = 0)
    std_on_pol_pretrain_3000_pre_20 = np.std(on_pol_pretrain_3000_pre_20_returns, axis = 0)
    mean_rand_pretrain_1000_pre_20 = np.mean(rand_pretrain_1000_pre_20_returns, axis = 0)
    std_rand_pretrain_1000_pre_20 = np.std(rand_pretrain_1000_pre_20_returns, axis = 0)
    mean_rand_pretrain_3000_pre_20 = np.mean(rand_pretrain_3000_pre_20_returns, axis = 0)
    std_rand_pretrain_3000_pre_20 = np.std(rand_pretrain_3000_pre_20_returns, axis = 0)

    mean_mix_7b_pre_20[:20] = Qwen_7B[:20]
    mean_mix_32b_pre_20[:20] = Qwen_32B[:20]
    mean_pretrain_32b_1000_pre_20[:20] = Qwen_32B[:20]
    mean_pretrain_7b_1000_pre_20[:20] = Qwen_7B[:20]
    mean_pretrain_32b_3000_pre_20[:20] = Qwen_32B[:20]
    mean_pretrain_7b_3000_pre_20[:20] = Qwen_7B[:20]

    mean_mix_32b_pre_30 = np.mean(mix_32b_pre_30_returns, axis = 0)
    std_mix_32b_pre_30 = np.std(mix_32b_pre_30_returns, axis = 0)
    mean_mix_7b_pre_30 = np.mean(mix_7b_pre_30_returns, axis = 0)
    std_mix_7b_pre_30 = np.std(mix_7b_pre_30_returns, axis = 0)
    mean_pretrain_32b_1000_pre_30 = np.mean(pretrain_32b_1000_pre_30_returns, axis = 0)
    std_pretrain_32b_1000_pre_30 = np.std(pretrain_32b_1000_pre_30_returns, axis = 0)
    mean_pretrain_7b_1000_pre_30 = np.mean(pretrain_7b_1000_pre_30_returns, axis = 0)
    std_pretrain_7b_1000_pre_30 = np.std(pretrain_7b_1000_pre_30_returns, axis = 0)
    mean_pretrain_32b_3000_pre_30 = np.mean(pretrain_32b_3000_pre_30_returns, axis = 0)
    std_pretrain_32b_3000_pre_30 = np.std(pretrain_32b_3000_pre_30_returns, axis = 0)
    mean_pretrain_7b_3000_pre_30 = np.mean(pretrain_7b_3000_pre_30_returns, axis = 0)
    std_pretrain_7b_3000_pre_30 = np.std(pretrain_7b_3000_pre_30_returns, axis = 0)
    mean_on_pol_pretrain_1000_pre_30 = np.mean(on_pol_pretrain_1000_pre_30_returns, axis = 0)
    std_on_pol_pretrain_1000_pre_30 = np.std(on_pol_pretrain_1000_pre_30_returns, axis = 0)
    mean_on_pol_pretrain_3000_pre_30 = np.mean(on_pol_pretrain_3000_pre_30_returns, axis = 0)
    std_on_pol_pretrain_3000_pre_30 = np.std(on_pol_pretrain_3000_pre_30_returns, axis = 0)
    mean_rand_pretrain_1000_pre_30 = np.mean(rand_pretrain_1000_pre_30_returns, axis = 0)
    std_rand_pretrain_1000_pre_30 = np.std(rand_pretrain_1000_pre_30_returns, axis = 0)
    mean_rand_pretrain_3000_pre_30 = np.mean(rand_pretrain_3000_pre_30_returns, axis = 0)
    std_rand_pretrain_3000_pre_30 = np.std(rand_pretrain_3000_pre_30_returns, axis = 0)

    mean_mix_7b_pre_30[:30] = Qwen_7B[:30]
    mean_mix_32b_pre_30[:30] = Qwen_32B[:30]
    mean_pretrain_32b_1000_pre_30[:30] = Qwen_32B[:30]
    mean_pretrain_7b_1000_pre_30[:30] = Qwen_7B[:30]
    mean_pretrain_32b_3000_pre_30[:30] = Qwen_32B[:30]
    mean_pretrain_7b_3000_pre_30[:30] = Qwen_7B[:30]

    out_dict = {
        "x": x,
        "mean_onl": mean_onl,
        "std_onl": std_onl,
        "Qwen_7B": Qwen_7B,
        "Qwen_32B": Qwen_32B,
        "mean_random": mean_random,
        "mean_mix_7b_pre_10": mean_mix_7b_pre_10,
        "mean_mix_32b_pre_10": mean_mix_32b_pre_10,
        "mean_pretrain_32b_1000_pre_10": mean_pretrain_32b_1000_pre_10,
        "mean_pretrain_7b_1000_pre_10": mean_pretrain_7b_1000_pre_10,
        "mean_pretrain_32b_3000_pre_10": mean_pretrain_32b_3000_pre_10,
        "mean_pretrain_7b_3000_pre_10": mean_pretrain_7b_3000_pre_10,
        "std_mix_32b_pre_10": std_mix_32b_pre_10,
        "std_mix_7b_pre_10": std_mix_7b_pre_10,
        "std_pretrain_32b_1000_pre_10": std_pretrain_32b_1000_pre_10,
        "std_pretrain_7b_1000_pre_10": std_pretrain_7b_1000_pre_10,
        "std_pretrain_32b_3000_pre_10": std_pretrain_32b_3000_pre_10,
        "std_pretrain_7b_3000_pre_10": std_pretrain_7b_3000_pre_10,
        "mean_on_pol_pretrain_1000_pre_10": mean_on_pol_pretrain_1000_pre_10,
        "std_on_pol_pretrain_1000_pre_10": std_on_pol_pretrain_1000_pre_10,
        "mean_on_pol_pretrain_3000_pre_10": mean_on_pol_pretrain_3000_pre_10,
        "std_on_pol_pretrain_3000_pre_10": std_on_pol_pretrain_3000_pre_10,
        "mean_rand_pretrain_1000_pre_10": mean_rand_pretrain_1000_pre_10,
        "std_rand_pretrain_1000_pre_10": std_rand_pretrain_1000_pre_10,
        "mean_rand_pretrain_3000_pre_10": mean_rand_pretrain_3000_pre_10,
        "std_rand_pretrain_3000_pre_10": std_rand_pretrain_3000_pre_10,
        "mean_mix_7b_pre_20": mean_mix_7b_pre_20, #20 here
        "mean_mix_32b_pre_20": mean_mix_32b_pre_20,
        "mean_pretrain_32b_1000_pre_20": mean_pretrain_32b_1000_pre_20,
        "mean_pretrain_7b_1000_pre_20": mean_pretrain_7b_1000_pre_20,
        "mean_pretrain_32b_3000_pre_20": mean_pretrain_32b_3000_pre_20,
        "mean_pretrain_7b_3000_pre_20": mean_pretrain_7b_3000_pre_20,
        "std_mix_32b_pre_20": std_mix_32b_pre_20,
        "std_mix_7b_pre_20": std_mix_7b_pre_20,
        "std_pretrain_32b_1000_pre_20": std_pretrain_32b_1000_pre_20,
        "std_pretrain_7b_1000_pre_20": std_pretrain_7b_1000_pre_20,
        "std_pretrain_32b_3000_pre_20": std_pretrain_32b_3000_pre_20,
        "std_pretrain_7b_3000_pre_20": std_pretrain_7b_3000_pre_20,
        "mean_on_pol_pretrain_1000_pre_20": mean_on_pol_pretrain_1000_pre_20,
        "std_on_pol_pretrain_1000_pre_20": std_on_pol_pretrain_1000_pre_20,
        "mean_on_pol_pretrain_3000_pre_20": mean_on_pol_pretrain_3000_pre_20,
        "std_on_pol_pretrain_3000_pre_20": std_on_pol_pretrain_3000_pre_20,
        "mean_rand_pretrain_1000_pre_20": mean_rand_pretrain_1000_pre_20,
        "std_rand_pretrain_1000_pre_20": std_rand_pretrain_1000_pre_20,
        "mean_rand_pretrain_3000_pre_20": mean_rand_pretrain_3000_pre_20,
        "std_rand_pretrain_3000_pre_20": std_rand_pretrain_3000_pre_20,
        "mean_mix_7b_pre_30": mean_mix_7b_pre_30, #30 here
        "mean_mix_32b_pre_30": mean_mix_32b_pre_30,
        "mean_pretrain_32b_1000_pre_30": mean_pretrain_32b_1000_pre_30,
        "mean_pretrain_7b_1000_pre_30": mean_pretrain_7b_1000_pre_30,
        "mean_pretrain_32b_3000_pre_30": mean_pretrain_32b_3000_pre_30,
        "mean_pretrain_7b_3000_pre_30": mean_pretrain_7b_3000_pre_30,
        "std_mix_32b_pre_30": std_mix_32b_pre_30,
        "std_mix_7b_pre_30": std_mix_7b_pre_30,
        "std_pretrain_32b_1000_pre_30": std_pretrain_32b_1000_pre_30,
        "std_pretrain_7b_1000_pre_30": std_pretrain_7b_1000_pre_30,
        "std_pretrain_32b_3000_pre_30": std_pretrain_32b_3000_pre_30,
        "std_pretrain_7b_3000_pre_30": std_pretrain_7b_3000_pre_30,
        "mean_on_pol_pretrain_1000_pre_30": mean_on_pol_pretrain_1000_pre_30,
        "std_on_pol_pretrain_1000_pre_30": std_on_pol_pretrain_1000_pre_30,
        "mean_on_pol_pretrain_3000_pre_30": mean_on_pol_pretrain_3000_pre_30,
        "std_on_pol_pretrain_3000_pre_30": std_on_pol_pretrain_3000_pre_30,
        "mean_rand_pretrain_1000_pre_30": mean_rand_pretrain_1000_pre_30,
        "std_rand_pretrain_1000_pre_30": std_rand_pretrain_1000_pre_30,
        "mean_rand_pretrain_3000_pre_30": mean_rand_pretrain_3000_pre_30,
        "std_rand_pretrain_3000_pre_30": std_rand_pretrain_3000_pre_30
    }
    try:
        for k,v in out_dict_SFT.items():
            out_dict[k] = v
    except:
        pass
    try:
        for k,v in out_dict_DS.items():
            out_dict[k] = v
    except:
        pass
    return out_dict

In [188]:
def plot_main(hyperparams, cache):
    plt.figure(figsize=(12, 6))
    plt.plot(cache['mean_onl'], label='On-policy')
    plt.fill_between(cache['x'], cache['mean_onl']-cache['std_onl']/np.sqrt(hyperparams["n_exp"]), cache['mean_onl']+cache['std_onl']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
    plt.plot(cache['mean_pretrain_7b_3000_pre_10'], label='LORO')
    plt.fill_between(cache['x'], cache['mean_pretrain_7b_3000_pre_10']-cache['std_pretrain_7b_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), cache['mean_pretrain_7b_3000_pre_10'] +cache['std_pretrain_7b_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)

    plt.plot(Qwen_7B, label='Qwen_7B', linestyle=':')
    plt.plot(Qwen_32B, label='Qwen_32B', linestyle=':')
    plt.plot(mean_random, label='Random', linestyle=':')

    plt.title(f'{hyperparams["env"].split("-")[0]} episode reward with std error for {hyperparams["n_exp"]} seeds.')
    plt.xlabel('# of episodes')
    plt.ylabel('Episode reward')
    plt.legend(loc='lower right')
    # plt.grid(True)
    # plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    if "Cliff" in hyperparams["env"]:
        plt.ylim(-500, 0)
    if "MountainCar" in hyperparams["env"]:
        plt.ylim(-250, -100)
    # plt.xticks(np.arange(0, 101, 10))
    plt.savefig(f'figs/{hyperparams["env"].split("-")[0]}_main.png', bbox_inches='tight', pad_inches=0.1)
    plt.show()

def plot_pretrain(hyperparams, cache):
    plt.figure(figsize=(12, 6))
    plt.plot(cache['mean_onl'], label='On-policy')
    plt.fill_between(cache['x'], cache['mean_onl']-cache['std_onl']/np.sqrt(hyperparams["n_exp"]), cache['mean_onl']+cache['std_onl']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
    plt.plot(cache['mean_pretrain_7b_3000_pre_10'], label='LORO')
    plt.fill_between(cache['x'], cache['mean_pretrain_7b_3000_pre_10']-cache['std_pretrain_7b_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), cache['mean_pretrain_7b_3000_pre_10'] +cache['std_pretrain_7b_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
    plt.plot(cache['mean_on_pol_pretrain_3000_pre_10'], label='Pretrain w/ On-policy data')
    plt.fill_between(cache['x'], cache['mean_on_pol_pretrain_3000_pre_10']-cache['std_on_pol_pretrain_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), cache['mean_on_pol_pretrain_3000_pre_10'] +cache['std_on_pol_pretrain_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
    plt.plot(cache['mean_rand_pretrain_3000_pre_10'], label='Pretrain w/ random policy data')
    plt.fill_between(cache['x'], cache['mean_rand_pretrain_3000_pre_10']-cache['std_rand_pretrain_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), cache['mean_rand_pretrain_3000_pre_10'] +cache['std_rand_pretrain_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)

    plt.title(f'{hyperparams["env"].split("-")[0]} episode reward with std error for {hyperparams["n_exp"]} seeds.')
    plt.xlabel('# of episodes')
    plt.ylabel('Episode reward')
    if "Cliff" in hyperparams["env"] or "MountainCar" in hyperparams["env"]:
        plt.ylim(-500, 0)
    if "MountainCar" in hyperparams["env"]:
        plt.ylim(-250, -100)
    plt.legend(loc='lower right')
    plt.savefig(f'figs/{hyperparams["env"].split("-")[0]}_pretrain.png', bbox_inches='tight', pad_inches=0.1)
    plt.show()

def plot_mix(hyperparams, cache):
    plt.figure(figsize=(12, 6))
    plt.plot(cache['mean_onl'], label='On-policy')
    plt.fill_between(cache['x'], cache['mean_onl']-cache['std_onl']/np.sqrt(hyperparams["n_exp"]), cache['mean_onl']+cache['std_onl']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
    plt.plot(cache['mean_pretrain_7b_3000_pre_10'], label='LORO')
    plt.fill_between(cache['x'], cache['mean_pretrain_7b_3000_pre_10']-cache['std_pretrain_7b_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), cache['mean_pretrain_7b_3000_pre_10']+cache['std_pretrain_7b_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
    plt.plot(cache['mean_mix_7b_pre_10'], label='Mix data w/o pretrain')
    plt.fill_between(cache['x'], cache['mean_mix_7b_pre_10']-cache['std_mix_7b_pre_10']/np.sqrt(hyperparams["n_exp"]), cache['mean_mix_7b_pre_10']+cache['std_mix_7b_pre_10']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)

    plt.title(f'{hyperparams["env"].split("-")[0]} episode reward with std error for {hyperparams["n_exp"]} seeds.')
    plt.xlabel('# of episodes')
    plt.ylabel('Episode reward')
    if "Cliff" in hyperparams["env"]:
        plt.ylim(-500, 0)
    if "MountainCar" in hyperparams["env"]:
        plt.ylim(-250, -100)
    plt.legend(loc='lower right')
    plt.savefig(f'figs/{hyperparams["env"].split("-")[0]}_mix.png', bbox_inches='tight', pad_inches=0.1)
    plt.show()

def plot_sft_lcot(hyperparams, cache):
    try:
        plt.figure(figsize=(12, 6))
        plt.plot(cache['mean_onl'], label='On-policy')
        plt.fill_between(cache['x'], cache['mean_onl']-cache['std_onl']/np.sqrt(hyperparams["n_exp"]), cache['mean_onl']+cache['std_onl']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
        plt.plot(cache['mean_pretrain_7b_3000_pre_10'], label='LORO')
        plt.fill_between(cache['x'], cache['mean_pretrain_7b_3000_pre_10']-cache['std_pretrain_7b_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), cache['mean_pretrain_7b_3000_pre_10']+cache['std_pretrain_7b_3000_pre_10']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
        plt.plot(cache['mean_pretrain_7b_3000_pre_10SFT'], label='SFT')
        plt.fill_between(cache['x'], cache['mean_pretrain_7b_3000_pre_10SFT']-cache['std_pretrain_7b_3000_pre_10SFT']/np.sqrt(hyperparams["n_exp"]), cache['mean_pretrain_7b_3000_pre_10SFT']+cache['std_pretrain_7b_3000_pre_10SFT']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)

        plt.title(f'{hyperparams["env"].split("-")[0]} episode reward with std error for {hyperparams["n_exp"]} seeds.')
        plt.xlabel('# of episodes')
        plt.ylabel('Episode reward')
        if "Cliff" in hyperparams["env"]:
            plt.ylim(-500, 0)
        if "MountainCar" in hyperparams["env"]:
            plt.ylim(-250, -100)
        plt.legend(loc='lower right')
    except:
        pass
    try:
        plt.plot(cache['mean_pretrain_7b_3000_pre_10DS'], label='Long-CoT 7B')
        plt.fill_between(cache['x'], cache['mean_pretrain_7b_3000_pre_10DS']-cache['std_pretrain_7b_3000_pre_10DS']/np.sqrt(hyperparams["n_exp"]), cache['mean_pretrain_7b_3000_pre_10DS']+cache['std_pretrain_7b_3000_pre_10DS']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
        plt.plot(cache['mean_pretrain_14b_3000_pre_10DS'], label='Long-CoT 14B')
        plt.fill_between(cache['x'], cache['mean_pretrain_14b_3000_pre_10DS']-cache['std_pretrain_14b_3000_pre_10DS']/np.sqrt(hyperparams["n_exp"]), cache['mean_pretrain_14b_3000_pre_10DS']+cache['std_pretrain_14b_3000_pre_10DS']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
    except:
        pass
    plt.savefig(f'figs/{hyperparams["env"].split("-")[0]}_sft_lcot.png', bbox_inches='tight', pad_inches=0.1)
    plt.show()

def plot_model_size(hyperparams, cache):
    fig, axs = plt.subplots(2, 3, figsize=(16, 10))
    for j in range(2):
        if j == 0:
            pretrain_step = 1000
        else:
            pretrain_step = 3000
        for i in range(3):
            mean_onl = cache[f'mean_onl']
            std_onl = cache[f'std_onl']/np.sqrt(hyperparams["n_exp"])
            mean_loro7 = cache[f'mean_pretrain_7b_{pretrain_step}_pre_{i+1}0']
            std_loro7 = cache[f'std_pretrain_7b_{pretrain_step}_pre_{i+1}0']/np.sqrt(hyperparams["n_exp"])
            mean_loro32 = cache[f'mean_pretrain_32b_{pretrain_step}_pre_{i+1}0']
            std_loro32 = cache[f'std_pretrain_32b_{pretrain_step}_pre_{i+1}0']/np.sqrt(hyperparams["n_exp"])
            axs[j,i].plot(mean_onl, label='On-policy')
            axs[j,i].fill_between(cache['x'], mean_onl-std_onl, mean_onl+std_onl, alpha=0.3)
            axs[j,i].plot(mean_loro7, label='LORO 7B')
            axs[j,i].fill_between(cache['x'], mean_loro7-std_loro7, mean_loro7+std_loro7, alpha=0.3)
            axs[j,i].plot(mean_loro32, label='LORO 32B')
            axs[j,i].fill_between(cache['x'], mean_loro32-std_loro32, mean_loro32+std_loro32, alpha=0.3)

            axs[j,i].set_title(f'Pretrain {i+1}0 episodes, for {pretrain_step} steps')
            axs[j,i].set_xlabel('# of episodes')
            axs[j,i].set_ylabel('Episode reward')
            if "Cliff" in hyperparams["env"]:
                axs[j,i].set_ylim(-500, 0)
            if "MountainCar" in hyperparams["env"]:
                axs[j,i].set_ylim(-250, -100)
    
    axs[0,0].legend(loc='lower right')
    plt.savefig(f'figs/{hyperparams["env"].split("-")[0]}_model_size.png', bbox_inches='tight', pad_inches=0.1)
    plt.show()

def plot_pretrain_step(hyperparams, cache):
    fig, axs = plt.subplots(2, 3, figsize=(16, 10))
    for j in range(2):
        if j == 0:
            model_size = 7
        else:
            model_size = 32
        for i in range(3):
            mean_onl = cache[f'mean_onl']
            std_onl = cache[f'std_onl']/np.sqrt(hyperparams["n_exp"])
            mean_loro_1k = cache[f'mean_pretrain_{model_size}b_1000_pre_{i+1}0']
            std_loro_1k = cache[f'std_pretrain_{model_size}b_1000_pre_{i+1}0']/np.sqrt(hyperparams["n_exp"])
            mean_loro_3k = cache[f'mean_pretrain_{model_size}b_3000_pre_{i+1}0']
            std_loro_3k = cache[f'std_pretrain_{model_size}b_3000_pre_{i+1}0']/np.sqrt(hyperparams["n_exp"])
            axs[j,i].plot(mean_onl, label='On-policy')
            axs[j,i].fill_between(cache['x'], mean_onl-std_onl, mean_onl+std_onl, alpha=0.3)
            axs[j,i].plot(mean_loro_1k, label='LORO 1k pretrain steps')
            axs[j,i].fill_between(cache['x'], mean_loro_1k-std_loro_1k, mean_loro_1k+std_loro_1k, alpha=0.3)
            axs[j,i].plot(mean_loro_3k, label='LORO 3k pretrain steps')
            axs[j,i].fill_between(cache['x'], mean_loro_3k-std_loro_3k, mean_loro_3k+std_loro_3k, alpha=0.3)

            axs[j,i].set_title(f'Pretrain {i+1}0 episodes, {model_size}B model.')
            axs[j,i].set_xlabel('# of episodes')
            axs[j,i].set_ylabel('Episode reward')
            if "Cliff" in hyperparams["env"]:
                axs[j,i].set_ylim(-500, 0)
            if "MountainCar" in hyperparams["env"]:
                axs[j,i].set_ylim(-250, -100)
    
    axs[0,0].legend(loc='lower right')
    plt.savefig(f'figs/{hyperparams["env"].split("-")[0]}_pretrain_step.png', bbox_inches='tight', pad_inches=0.1)
    plt.show()

def plot_pretrain_eps(hyperparams, cache):
    fig, axs = plt.subplots(2, 2, figsize=(16, 10))
    for j in range(2):
        if j == 0:
            model_size = 7
        else:
            model_size = 32
        for i in range(2):
            if i == 0:
                pretrain_step = 1000
            else:
                pretrain_step = 3000
            mean_onl = cache[f'mean_onl']
            std_onl = cache[f'std_onl']/np.sqrt(hyperparams["n_exp"])
            mean_loro_10 = cache[f'mean_pretrain_{model_size}b_{pretrain_step}_pre_10']
            std_loro_10 = cache[f'std_pretrain_{model_size}b_{pretrain_step}_pre_10']/np.sqrt(hyperparams["n_exp"])
            mean_loro_20 = cache[f'mean_pretrain_{model_size}b_{pretrain_step}_pre_20']
            std_loro_20 = cache[f'std_pretrain_{model_size}b_{pretrain_step}_pre_20']/np.sqrt(hyperparams["n_exp"])
            mean_loro_30 = cache[f'mean_pretrain_{model_size}b_{pretrain_step}_pre_30']
            std_loro_30 = cache[f'std_pretrain_{model_size}b_{pretrain_step}_pre_30']/np.sqrt(hyperparams["n_exp"])
            axs[j,i].plot(mean_onl, label='On-policy')
            axs[j,i].fill_between(cache['x'], mean_onl-std_onl, mean_onl+std_onl, alpha=0.3)
            axs[j,i].plot(mean_loro_10, label='LORO 10 pretrain eps')
            axs[j,i].fill_between(cache['x'], mean_loro_10-std_loro_10, mean_loro_10+std_loro_10, alpha=0.3)
            axs[j,i].plot(mean_loro_20, label='LORO 20 pretrain eps')
            axs[j,i].fill_between(cache['x'], mean_loro_20-std_loro_20, mean_loro_20+std_loro_20, alpha=0.3)
            axs[j,i].plot(mean_loro_30, label='LORO 30 pretrain eps')
            axs[j,i].fill_between(cache['x'], mean_loro_30-std_loro_30, mean_loro_30+std_loro_30, alpha=0.3)

            axs[j,i].set_title(f'Pretrain {pretrain_step} steps, {model_size}B model.')
            axs[j,i].set_xlabel('# of episodes')
            axs[j,i].set_ylabel('Episode reward')
            if "Cliff" in hyperparams["env"]:
                axs[j,i].set_ylim(-500, 0)
            if "MountainCar" in hyperparams["env"]:
                axs[j,i].set_ylim(-250, -100)
    
    axs[0,0].legend(loc='lower right')
    plt.savefig(f'figs/{hyperparams["env"].split("-")[0]}_pretrain_eps.png', bbox_inches='tight', pad_inches=0.1)
    plt.show()

def plot_pretrain_big(hyperparams, cache):
    fig, axs = plt.subplots(4, 3, figsize=(16, 20))
    for j in range(4):
        if j<2:
            model_size = 7
        else:
            model_size = 32
        if j%2 == 0:
            pretrain_step = 1000
        else:
            pretrain_step = 3000
        for i in range(3):
            mean_loro = cache[f'mean_pretrain_{model_size}b_{pretrain_step}_pre_{i+1}0']
            std_loro = cache[f'std_pretrain_{model_size}b_{pretrain_step}_pre_{i+1}0']/np.sqrt(hyperparams["n_exp"])
            mean_on_pol = cache[f'mean_on_pol_pretrain_{pretrain_step}_pre_{i+1}0']
            std_on_pol = cache[f'std_on_pol_pretrain_{pretrain_step}_pre_{i+1}0']/np.sqrt(hyperparams["n_exp"])
            mean_rand = cache[f'mean_rand_pretrain_{pretrain_step}_pre_{i+1}0']
            std_rand = cache[f'std_rand_pretrain_{pretrain_step}_pre_{i+1}0']/np.sqrt(hyperparams["n_exp"])
            axs[j,i].plot(cache['mean_onl'], label='On-policy')
            axs[j,i].fill_between(cache['x'], cache['mean_onl']-cache['std_onl']/np.sqrt(hyperparams["n_exp"]), cache['mean_onl']+cache['std_onl']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
            axs[j,i].plot(mean_loro, label='LORO')
            axs[j,i].fill_between(cache['x'], mean_loro-std_loro, mean_loro+std_loro, alpha=0.3)
            axs[j,i].plot(mean_on_pol, label='Pretrain w/ On-policy data')
            axs[j,i].fill_between(cache['x'], mean_on_pol-std_on_pol, mean_on_pol+std_on_pol, alpha=0.3)
            axs[j,i].plot(mean_rand, label='Pretrain w/ random policy data')
            axs[j,i].fill_between(cache['x'], mean_rand-std_rand, mean_rand+std_rand, alpha=0.3)

            axs[j,i].set_title(f'Pretrain {i+1}0 episodes, {model_size}B model, for {pretrain_step} steps')
            axs[j,i].set_xlabel('# of episodes')
            axs[j,i].set_ylabel('Episode reward')
            if "Cliff" in hyperparams["env"]:
                axs[j,i].set_ylim(-500, 0)
            if "MountainCar" in hyperparams["env"]:
                axs[j,i].set_ylim(-250, -100)
    
    axs[0,0].legend(loc='lower right')
    plt.savefig(f'figs/{hyperparams["env"].split("-")[0]}_pretrain_big.png', bbox_inches='tight', pad_inches=0.1)
    plt.show()

def plot_sft_lcot_big(hyperparams, cache):
    fig, axs = plt.subplots(2, 3, figsize=(16, 10))
    for j in range(2):
        if j == 0:
            pretrain_step = 1000
        else:
            pretrain_step = 3000
        for i in range(3):
            try:
                mean_loro = cache[f'mean_pretrain_7b_{pretrain_step}_pre_{i+1}0']
                std_loro = cache[f'std_pretrain_7b_{pretrain_step}_pre_{i+1}0']/np.sqrt(hyperparams["n_exp"])
                mean_sft = cache[f'mean_pretrain_7b_{pretrain_step}_pre_{i+1}0SFT']
                std_sft = cache[f'mean_pretrain_7b_{pretrain_step}_pre_{i+1}0SFT']/np.sqrt(hyperparams["n_exp"])
                axs[j,i].plot(cache['mean_onl'], label='On-policy')
                axs[j,i].fill_between(cache['x'], cache['mean_onl']-cache['std_onl']/np.sqrt(hyperparams["n_exp"]), cache['mean_onl']+cache['std_onl']/np.sqrt(hyperparams["n_exp"]), alpha=0.3)
                axs[j,i].plot(mean_loro, label='LORO')
                axs[j,i].fill_between(cache['x'], mean_loro-std_loro, mean_loro+std_loro, alpha=0.3)
                axs[j,i].plot(mean_sft, label='SFT')
                axs[j,i].fill_between(cache['x'], mean_sft-std_sft, mean_sft+std_sft, alpha=0.3)

                axs[j,i].set_title(f'Pretrain {i+1}0 episodes, 7B model, for {pretrain_step} steps')
                axs[j,i].set_xlabel('# of episodes')
                axs[j,i].set_ylabel('Episode reward')
                if "Cliff" in hyperparams["env"]:
                    axs[j,i].set_ylim(-500, 0)
                if "MountainCar" in hyperparams["env"]:
                    axs[j,i].set_ylim(-250, -100)
            except:
                pass
            try:
                mean_ds7 = cache[f'mean_pretrain_7b_{pretrain_step}_pre_{i+1}0DS']
                std_ds7 = cache[f'mean_pretrain_7b_{pretrain_step}_pre_{i+1}0DS']/np.sqrt(hyperparams["n_exp"])
                mean_ds14 = cache[f'mean_pretrain_14b_{pretrain_step}_pre_{i+1}0DS']
                std_ds14 = cache[f'mean_pretrain_14b_{pretrain_step}_pre_{i+1}0DS']/np.sqrt(hyperparams["n_exp"])
                axs[j,i].plot(mean_ds7, label='Long-CoT 7B')
                axs[j,i].fill_between(cache['x'], mean_ds7-std_ds7, mean_ds7+std_ds7, alpha=0.3)
                axs[j,i].plot(mean_ds14, label='Long-CoT 14B')
                axs[j,i].fill_between(cache['x'], mean_ds14-std_ds14, mean_ds14+std_ds14, alpha=0.3)
            except:
                pass
    
    axs[0,0].legend(loc='lower right')
    plt.savefig(f'figs/{hyperparams["env"].split("-")[0]}_sft_lcot_bigs.png', bbox_inches='tight', pad_inches=0.1)
    plt.show()

In [189]:
all_cache = extract_data(hyperparams, Qwen_7B, Qwen_32B, DS_7B, DS_14B, Qwen_7B_SFT)
plot_main(hyperparams, all_cache)
plot_pretrain(hyperparams, all_cache)
# plot_pretrain_big(hyperparams, all_cache)
plot_mix(hyperparams, all_cache)
plot_model_size(hyperparams, all_cache)
plot_pretrain_step(hyperparams, all_cache)
plot_pretrain_eps(hyperparams, all_cache)
# plot_sft_lcot(hyperparams, all_cache)
plot_sft_lcot_big(hyperparams, all_cache)

FileNotFoundError: [Errno 2] No such file or directory: 'data/cache_RepresentedPong_on_policy_pretrain_exp_rand.pkl'